Scroll a specific element?


In [5]:
function scrollClient(selector, up = false) {
    // scroll to bottom of messages
    return client
        .execute((selector, up = false) => {
            if (selector === '' || selector === false) {
                window.scroll(window.scrollX, window.scrollY + (up ? -100000 : 100000));
                return;
            }
            var people = document.evaluate(
                selector,
                document, null,
                XPathResult.FIRST_ORDERED_NODE_TYPE, null).singleNodeValue;
            people.scrollTop = people.scrollTop + (up ? -100000 : 100000);
        }, selector, up)
};
module.exports = scrollClient;
scrollClient;


[Function: scrollClient]

Get all elements XPath?



In [3]:
function getAllXPathBrowser(selector) {
    var getArrayXPath = (selector, ctx, index) => {
        if (typeof ctx == 'undefined') {
            ctx = module.exports.document;
        }
        var iterator = module.exports.document.evaluate(
            selector.replace('$i', index),
            ctx, null,
            (XPathResult || {}).ORDERED_NODE_ITERATOR_TYPE || 5, null);
        var co = [];
        var m;
        while ((m = iterator.iterateNext()) && co.push(m.nodeValue || m)) ;
        return co;
    };

    var getObjectXPath = (object, ctx, index) => {
        var result = {};
        // TODO: parse object
        for (var p in object) {
            if (object.hasOwnProperty(p)) {
                result[p] = resolveXPathQueries(object[p], ctx, index);
            }
        }
        return result;
    }

    var resolveXPathQueries = (selector, ctx, index) => {
        var inputArr = selector;
        if (!Array.isArray(inputArr)) {
            inputArr = [inputArr];
        }
        // parse array of string and objects
        var result = [], previousResults;
        for (var i = 0; i < inputArr.length; i++) {
            if (typeof inputArr[i] == 'string') {
                if (typeof previousResults != 'undefined') {
                    result = result.concat(previousResults);
                }
                // store in case the next query is a mapped object
                previousResults = getArrayXPath(inputArr[i], ctx, index);
            } else if (typeof inputArr[i] == 'object') {
                if (typeof previousResults !== 'undefined') {
                    previousResults = previousResults.map((p, j) =>
                        getObjectXPath(inputArr[i], p, j));
                    result = result.concat(previousResults);
                    previousResults = undefined;
                } else {
                    result[result.length] = getObjectXPath(inputArr[i], ctx, index);
                }
            } else if (Array.isArray(inputArr[i])) {
                result = result.concat(resolveXPathQueries(inputArr[i], ctx));
            } else if (typeof inputArr[i] == 'function') {
                // TODO: what to do here?
            }
        }
        if (typeof previousResults != 'undefined') {
            result = result.concat(previousResults);
        }

        if (!Array.isArray(selector) && result.length == 1) {
            return result[0];
        }
        return result;
    };
    return resolveXPathQueries(selector);
}

// TODO: better way to do this for selenium versus browser?
function getAllXPath(selector) {
    return module.exports.client.execute(getAllXPathBrowser, selector)
        .then(r => r.value)
        .catch(e => console.log(e));
}

if (typeof client !== 'undefined' && typeof client.getAllXPath === 'undefined') {
    client.addCommand('getAllXPath', getAllXPath);
}
module.exports = {
    getAllXPath,
    client
}


[Function: getAllXPath]

Get all elements until?



In [7]:
var importer = require('../Core');
var {
    getAllXPath, scrollClient
} = importer.import([
    'all elements xpath',
    'scroll specific element'
], {client})

function getAllUntil(scrollableSelector,
                     dataSelector,
                     set = [],
                     compare = (a, b) => a === b,
                     cb = (i) => i < 3,
                     up = false,
                     i = 0) {
    return getAllXPath(dataSelector)
        .then(r => {
            var newPosts = ((typeof r === 'string' ? [r] : r) || [])
                .filter(e => set
                    .filter(m => compare(e, m)).length === 0);
            set = newPosts.concat(set);
            return newPosts.length > 0
                ? scrollClient(scrollableSelector, up)
                    .pause(4000)
                    .then(() => cb(i)
                        ? getAllUntil(scrollableSelector, dataSelector, set, compare, cb, up, i + 1)
                        : set)
                : Promise.resolve(set);
        })
        .catch(e => console.log(e))
};
if (typeof client.getAllUntil == 'undefined') {
    client.addCommand('getAllUntil', getAllUntil);
}
module.exports = getAllUntil;




[Function: getAllUntil]

click spa link?


In [ ]:
function clickSpa(profile) {
    return client
        .getUrl()
        .then(url => url.indexOf(profile) > -1
              ? []
              : client
             .execute(url => {
                var a = document.createElement('a');
                a.setAttribute('href', url);
                a.setAttribute('class', 'spa');
                a.onclick = () => setTimeout(() => document.body.removeChild(a), 500)
                a.href = url;
                a.style.position = 'absolute';
                a.style.zIndex = 4294967295;
                a.style.top = 0;
                a.style.left = 0;
                a.style.bottom = 0;
                a.style.right = 0;
                document.body.appendChild(a);
            }, profile)
        .click('a.spa[href*="' + profile + '"]')
        .pause(1000))
        .catch(e => console.log(e))
};
if (typeof client.clickSpa == 'undefined') {
    client.addCommand('clickSpa', clickSpa);
}
module.exports = clickSpa;


xpath minimizing service from recording app



In [ ]:


static getArrayXPath(selector: string, ctx?: Node): Array<Node> {
    if (typeof ctx === 'undefined') {
        ctx = document;
    }
    const iterator = document.evaluate(
            selector,
            ctx, void 0,
            XPathResult.ORDERED_NODE_ITERATOR_TYPE, void 0);
    const co: Array<Node> = [];
    let m = iterator.iterateNext();
    while (m) {
        co.push(m);
        m = iterator.iterateNext();
    }

    return co;
}

static convertXPathToCss(path: string): string {
    // if matched xpath is simple enough, convert it to CSS
    // DIV[contains(@class, "product-tile")]/parent::*/DIV[2]//MD-CARD[contains(@class, "mat-card")]/parent::*/MD-CARD[1]
    return path
            .replace(/\/([a-z-]+)\[@id="(.*?)"]/ig, '/$1#$2')
            .replace(/\/([^\/]+)\[contains\(@class, "(.*?)"\)]/ig, '/$1.$2')
            .replace(/\/parent::[a-z-]+\/[a-z-]+\[([0-9]+)]/ig, ':nth-child($1)')
            .replace(/^\/\//ig, '')
            .replace(/\/\//ig, ' ')
            .replace(/\//ig, ' > ');
}

elementToXPath(path: Array<EventTarget>): string {

    // algorithm magic!
    // create an very specific XPath following these rules
    const addSegment = (elem: Node): string => {
        if (typeof elem === 'undefined' || elem === null) {
            return '/';
        }
        // add:
        // * element tag name,
        const tag = elem.nodeName;
        // * ids,
        let id: string | boolean = false;
        if (typeof (elem as any).getAttribute === 'function') {
            id = (elem as any).getAttribute('id');
        }
        // * classes with nouns (extra credit dictionary or service integration?),
        let className: string | boolean = false;
        if (typeof (elem as any).classList !== 'undefined') {
            className = (elem as any).classList[ 0 ];
        }
        // * indexes when all siblings are homogeneous, i.e. <li> and <tr><td>
        let countTypes = 0;
        if (typeof elem.parentElement !== 'undefined' && elem.parentElement !== null) {
            for (let e = 0; e < elem.parentElement.children.length; e++) {
                const el = elem.parentElement.children[ e ];
                if (el.nodeName === elem.nodeName) {
                    countTypes += 1;
                }
                if (el === elem) {
                    break;
                }
            }
        }

        return `${tag
                ? tag : ''}${id
                ? `[@id="${id}"]` : ''}${className
                ? `[contains(@class, "${className}")]` : ''}${countTypes > 0
                ? `/parent::${elem.parentElement.nodeName}/${tag}[${countTypes}]` : ''}`;
    };
    // use a stack to start for easy debugging
    const pathStack = [];
    // as we traverse parents of evt.target DOM element
    for (const currentEl of path) {
        pathStack.unshift(addSegment(currentEl as Node));
        if ((currentEl as Node).nodeName === 'BODY') {
            break;
        }
        if ((currentEl as Node).nodeName === 'APP-ROOT') {
            return;
        }
    }

    // flatten the XPath stack using '/'
    const fullPath = '//*[not(./app-root)]//' + pathStack.join('/');
    const matches = RecordingComponent.getArrayXPath(fullPath);
    if (matches.length !== 1) {
        throw new Error('Can\'t do anything right!');
    }

    // minimize XPath by splitting up every combination and checking only 1 element is matched for click
    const combinations: Array<string> = [];
    // * between 1 to N segments
    for (let i = 1; i < pathStack.length; i++) {
        for (let j = 1; j < i; j++) {
            const segments = ([
                ...pathStack.slice(j, i),
                // replace match with an extra / to create // between unnecessary segments
                '', // added to create // between target element
                pathStack[ pathStack.length - 1 ]
            ]).join('/');
            combinations.push('//' + segments);
        }
        // * starts from 0 to N
        // *
    }

    // sort by smallest and return shortest path matching 1 DOM element
    const minimal = combinations
            .filter(c => RecordingComponent.getArrayXPath(c).length === 1)
            .sort((a, b) => a.length - b.length);

    return minimal[ 0 ];
    // TODO: prioritize by button/input, ids, classes, attributes? (extra credit), index
}


Find the element rect dimensions that match this OCR detection of text content on page:

https://www.mathworks.com/help/vision/examples/automatically-detect-and-recognize-text-in-natural-images.html?s_tid=gn_loc_drop

